In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from script.tools import tools
import sys
print(sys.executable)
import net
import os
from script.tools import collection

print(collection)
result = collection.collection("collection/workspace1")
result = collection.collection("collection/workspace2",result)

/opt/homebrew/anaconda3/bin/python
<module 'script.tools.collection' from '/Users/ezra/worksapce/MQSim/script/tools/collection.py'>
472
1774


In [7]:
print(len(result))

1774


In [38]:
my_list = [[pow(2,32), 2, 3, 4, 5],[1,2,3,4,6]]

# 将列表转换为张量
my_tensor = torch.tensor(my_list,dtype=torch.float)
print(my_tensor)
print(type(my_tensor))
m = net.net(5, 4,20, 1)
out = m(my_tensor)
print(out)

tensor([[4.2950e+09, 2.0000e+00, 3.0000e+00, 4.0000e+00, 5.0000e+00],
        [1.0000e+00, 2.0000e+00, 3.0000e+00, 4.0000e+00, 6.0000e+00]])
<class 'torch.Tensor'>
4
tensor([[-0.0323],
        [-0.0269]], grad_fn=<AddmmBackward0>)
